# Escasez de Agua y el Impacto en el Valor de las Propiedades en Ciudad de México

* Author : Axel Daniel Malváez Flores
* Date : 2025-06-26
* Motivación : Análisis que tiene como principal objetivo estudiar la relación entre la escasez de agua en Ciudad de México y el impacto en el valor de las propiedades inmobiliarias. La escasez de agua ya no es solo un problema ambiental, sino un riesgo financiero y urbano tangible. El real estate, como sector clave en el desarrollo económico y el uso del suelo, debe adaptarse a este nuevo paradigma, integrando métricas de sostenibilidad hídrica en sus decisiones de inversión, construcción y planeación.

## Story Telling

* Motivación principal : Uso personal, para mis padres y para mis amigos que desean comprar/rentar una propiedad en la Ciudad de México, y que desean entender cómo la escasez de agua puede afectar el valor de las propiedades y sus decisiones de compra o alquiler.
* Público objetivo : Personas interesadas en el sector inmobiliario, compradores y arrendatarios, desarrolladores inmobiliarios, urbanistas, y cualquier persona interesada en entender el impacto de la escasez de agua en el mercado inmobiliario de la Ciudad de México.

### **Introducción**   

La Ciudad de México, es una metrópoli que se encuentra en un constante crecimiento poblacional y de urbanización. Sin embargo este crecimiento ha traído consigo desafíos críticos como lo es la escasez de agua. Este es un problema que afecta la calidad de vida de los habitantes, no obstante el impacto puede verse más alla de lo social y ambiental, pues también puede verse reflejado en el valor de las propiedades inmobiliarias. A medida que la disponibilidad de agua se convierte en un factor determinante para la habitabilidad y sostenibilidad de las zonas urbanas, es crucial entender cómo esta escasez influye en el mercado inmobiliario. A medida que la disponibilidad de agua disminuye, los desarrolladores y los compradores deben reconsiderar el valor de las propiedades, llevando a una reevaluación del mercado inmobiliario en las áreas afectadas. 

Este análisis busca explorar esta relación, proporcionando una visión integral de cómo la crisis hídrica afecta no solo a los residentes, sino también a los inversores y desarrolladores en el sector inmobiliario.

### **Objetivos**   
* Otorgar un enfoque práctico para que principalmente compradores, inversionistas, arreendatarios y desarrolladores puedan tomar decisiones informadas sobre la compra, alquiler y desarrollo de propiedades en áreas afectadas por la escasez de agua.
* Analizar la relación entre la escasez de agua y el valor de las propiedades en Ciudad de México.
* Evaluar cómo la crisis hídrica afecta las decisiones de inversión y desarrollo inmobiliario.
* Proporcionar recomendaciones para integrar la sostenibilidad hídrica en las estrategias de desarrollo urbano.
* Identificar áreas de oportunidad para el sector inmobiliario en el contexto de la escasez de agua.


### **Metodología** 
  
Para llevar a cabo este análisis, se utilizarán diversas fuentes de datos, incluyendo estadísticas sobre el suministro de agua, precios de propiedades, y estudios de caso sobre desarrollos inmobiliarios en áreas afectadas por la escasez de agua. Se tomaran recursos de entrevistas y reportes de expertos en el sector inmobiliario y sostenibilidad, así como encuestas a residentes para obtener una perspectiva más amplia sobre el impacto de la escasez de agua en sus decisiones de compra y alquiler. Utilizaré modelos estadísticos para correlacionar la disponibilidad de agua con los precios de las propiedades en diferentes zonas de la ciudad, y se realizarán análisis geoespaciales para identificar patrones y tendencias en el mercado inmobiliario en relación con la crisis hídrica.

### **Resultados Esperados**   
* Identificar patrones claros entre la escasez de agua y la disminución del valor de las propiedades en áreas específicas de Ciudad de México.
* Identificar áreas de la ciudad donde la escasez de agua ha llevado a una disminución significativa en los precios de las propiedades derivado de la crisis hídrica.
* Proporcionar recomendaciones prácticas para desarrolladores y compradores sobre cómo la sostenibilidad hídrica puede influir en las decisiones de inversión y desarrollo inmobiliario.
* Contribuir a una mayor conciencia sobre la importancia de la sostenibilidad hídrica en el sector inmobiliario y su impacto en el valor de las propiedades en Ciudad de México.

### **Scientific Methodology**

* ✅ **Observación**
    * He notado que la escasez de agua en Ciudad de México ha aumentado en los últimos años, afectando tanto a los residentes, como a los desarrollos inmobiliarios.
* ✅ **Pregunta**
    * ¿La escasez del agua afecta las decisiones de compra y alquiler de propiedades en Ciudad de México?
    * ¿Cómo influye la disponibilidad de agua en el valor de las propiedades inmobiliarias en áreas afectadas por la crisis hídrica?
    * ¿Qué opciones tienen los desarrolladores y compradores para mitigar este impacto? 
    * ¿Pueden los desarrolladores mantener o incluso aumentar el valor de sus proyectos con opciones sustentables?

* ✅ **Hipótesis**
    * Encontrar patrones entre la escasez del agua, el valor de las propiedades en áreas afectadas por la crisis hídrica, densidad poblacional, y las decisiones de compra y alquiler de propiedades.
    * La escasez de agua afecta negativamente el valor de las propiedades en áreas específicas de Ciudad de México, llevando a una disminución en los precios de las propiedades y afectando las decisiones de compra y alquiler de los residentes.
    * La escasez de agua disminuye el valor de las propiedades en áreas afectadas, y los desarrolladores que integran soluciones sostenibles hídricas pueden mantener o incluso aumentar el valor de sus proyectos.

* ❓ **Experimentación**
    * Se recopilarán datos sobre el suministro de agua, precios de propiedades, y se revisaran estudios de caso sobre desarrollos inmobiliarios en áreas afectadas por la escasez de agua para analizar la relación entre estos factores.

* ❌**Análisis de Datos**
    * Se utilizarán modelos estadísticos y análisis geoespaciales para correlacionar la disponibilidad de agua con los precios de las propiedades en diferentes zonas de la ciudad.

* ❌**Conclusión**
    * Se espera que los resultados muestren una correlación negativa entre la escasez de agua y el valor de las propiedades en áreas específicas de Ciudad de México, y que los desarrolladores que implementan soluciones sostenibles hídricas puedan mitigar este impacto, contribuyendo a un mercado inmobiliario más resiliente y sostenible.

* ❌**Revisión por Pares**
    * Los hallazgos serán revisados por expertos en el sector inmobiliario y sostenibilidad para validar las conclusiones y recomendaciones.

* ❌**Publicación**
    * Los resultados se publicarán en un informe detallado y se compartirán con la comunidad inmobiliaria, urbanistas, y el público en general para fomentar una mayor conciencia sobre la importancia de la sostenibilidad hídrica en el sector inmobiliario.

* ❌**Seguimiento**
    * Se propondrá un seguimiento a largo plazo para evaluar el impacto de las recomendaciones implementadas y la evolución del mercado inmobiliario en relación con la escasez de agua.



### **Referencias**

* Comisión Nacional del Agua (CONAGUA). (2023). Informe sobre la disponibilidad de agua en Ciudad de México.
* Instituto Nacional de Estadística y Geografía (INEGI). (2023). Estadísticas del mercado inmobiliario en Ciudad de México.
* Secretaría de Desarrollo Urbano y Vivienda (SEDUVI). (2023). Plan de Desarrollo Urbano de Ciudad de México.
* Expertos en sostenibilidad hídrica y desarrollo urbano, entrevistas realizadas en 2024.
* Estudios de caso sobre desarrollos inmobiliarios sostenibles en áreas afectadas por la escasez de agua, incluyendo análisis de precios y estrategias implementadas.
* Encuestas a residentes sobre la percepción del valor de las propiedades en relación con la disponibilidad de agua, realizadas en 2024.
* Artículos académicos y reportes sobre el impacto de la escasez de agua en el mercado inmobiliario, incluyendo análisis de tendencias y patrones en diferentes ciudades del mundo.
* Informes de organizaciones no gubernamentales y grupos de investigación sobre la crisis hídrica en Ciudad de México y su impacto en las comunidades y el desarrollo urbano.

## Notas

Real Estate : 
* Dinámica de compraventa (alta rotación vs. estancamiento)
* Tiempos promedio de venta

## Workflow

* Exploratory Data Analysis (EDA)
* Data Cleaning
* Data Visualization
* Statistical Analysis
* Machine Learning Models
    * Predict the probability of drought in Mexico City counties
    * Model Evaluation
* Conclusion
* Recommendations
* References

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json
import geopandas as gpd

# Treemap visualization
import squarify

# Combinations and filtering
from itertools import combinations

import warnings
warnings.filterwarnings('ignore')

## Data

### Water

In [ ]:
# Daata consumption during the whole 2019 by neighborhood
data_consumo_19 = pd.read_csv('data/water/consumo/consumo_agua_historico_2019.csv')
data_consumo_19.drop_duplicates(inplace=True)

# Municipalities with drought data and higher/lower probability of drought
data_drought = pd.read_excel('data/water/sequia/Municipios_con_ sequia.xlsx', sheet_name='MONITOR - SEMAFORO - USO EFIC')
cleaned_cols = data_drought.columns.to_series().where(~data_drought.columns.str.contains('^Unnamed'), '')
data_drought.columns = [x + '/' + y if x != '' else y for x, y in zip(cleaned_cols, data_drought.iloc[0].astype(str))]
data_drought = data_drought.iloc[1:].reset_index().iloc[:,1:]

# Water-related reports for 2022-2024
data_water_reps_22_24 = pd.read_csv('data/water/reportes/reportes_agua_2024_01.csv')#, parse_dates=['fecha_referencia'])
data_water_reps_hist = pd.read_csv('data/water/reportes/reportes_agua_hist.csv')#, parse_dates=['fecha_referencia'])

# Hidric feasibility
with open("data/water/factibilidad/factibilidad-hdrica.json", "r", encoding="utf-8") as f:
    data = json.load(f)
data_feasibility = gpd.GeoDataFrame.from_features(data["features"])

# Consumption every two months by neighborhood
with open('data/water/consumo/consumo-habitacional-promedio-bimestral-de-agua-por-colonia-m3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
hab_consoumption = gpd.GeoDataFrame.from_features(data['features'])

#### Data Dictionary

In [ ]:
data_dict = pd.read_csv('data/water/consumo/diccionario_consumo_hab_prom_agua_col.csv', encoding='iso-8859-1')
data_dict = data_dict[['Etiqueta del campo o variable', 'Descripció de Campo']]
data_dict
data_dict.loc[10]['Descripció de Campo']

### Property Prices & Population Statistics

In [ ]:
data_concentracion = gpd.read_file("data/property-population/alta_concentracion/zonas_vivienda.shp")
data_idx_SHF = pd.read_csv('data/property-population/SHF/indice_SHF.csv', encoding='iso-8859-1', delimiter=';')
data_leed = pd.read_excel('data/property-population/Leed/PublicLEEDProjectDirectory.xlsx')
data_tasa_crecimiento_alcaldia = pd.read_csv('data/property-population/CrecimientoPoblacional/poblacion_total_tasa_crecimiento_alcaldia_1.2.csv', encoding='utf-8')
data_zonas_valor = gpd.read_file("data/property-population/vus_promedio/valores_unitarios.shp")
data_hogares_col = gpd.read_file("data/property-population/Hogares por colonia/hogares_colonia.shp")

#### Data Dictionary

## Exploratory Data Analysis (EDA)

### Consumo de agua en Ciudad de México 2019

**Unidades**   
Medidas expresadas en  m³ de manera bimestral

**Descripción**
Para comenzar a desarrollar una historia podemos comenzar con el consumo de agua en Ciudad de México en 2019. Este es un punto de partida para entender la situación del agua en la ciudad y cómo ha evolucionado a lo largo del tiempo, sin necesidad de entrar en detalles técnicos complejos aún. Con estos datos podemos entender en el 2019 : 

* Cuáles eran las colonias con mayor consumo de agua, y cómo esto puede afectar el valor de las propiedades en esas áreas. 
* Qué tan desarrolladas son esas colonias y cómo influye en el consumo de agua.
* Qué tipo de inmuebles predominan en esas colonias y cómo esto puede relacionarse con el consumo de agua.

**Argumento**
* Refleja la presión real sobre la disponibilidad del recurso.
* La escasez hídrica ocurre por la suma de todos los usos: doméstico, comercial, industrial, institucional, etc.
* Algunas zonas con bajo consumo doméstico pueden estar tensionadas por altos consumos industriales y por lo tanto afectar el valor de las propiedades en esas áreas o si no, entender la afectación a futuro.

#### Preguntas


**Principal**:
* ¿Cuáles son las colonias con mayor y menor consumo de agua? - Usar total agregando por colonia y alcaldía
    * ¿Cuál es el la proporción del índice de desarrollo en dichas colonias?
    * ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?
    * ¿Hay colonias con un consumo desproporcionado en relación con el número de inmuebles? ¿Cuáles? - Usar total agregando por colonia y alcaldía

**Secundarias**:
* ¿Qué alcaldías tienen el mayor y menor consumo de agua? - Usar total agregando por colonia y alcaldía
    * ¿Cuál es el la proporción del índice de desarrollo en dichas alcaldías?
    * ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?
    * ¿Hay alcaldías con un consumo desproporcionado en relación con el número de inmuebles? - Usar total agregando por colonia y alcaldía

**External Data Cross-Referencing**:    
* ¿Hay colonias con un consumo desproporcionado en relación a su población?
* ¿Cómo se relaciona el consumo de agua con el valor de las propiedades en esas colonias

#### Insights

**Puntuales**    
* Colonias con mayor y menor consumo de agua
    * Colonias con mayor consumo de agua y menor índice de desarrollo - Alto riesgo de escasez de agua
    * Colonias con mayor consumo de agua y mayor índice de desarrollo - Alto riesgo de escasez de agua
    * Colonias con menor consumo de agua y menor índice de desarrollo - Bajo riesgo de escasez de agua
    * Colonias con menor consumo de agua y mayor índice de desarrollo - Bajo riesgo de escasez de agua


* Alcaldías con mayor y menor consumo de agua
    * Alcaldías con mayor consumo de agua y menor índice de desarrollo - Alto riesgo de escasez de agua
    * Alcaldías con mayor consumo de agua y mayor índice de desarrollo - Alto riesgo de escasez de agua
    * Alcaldías con menor consumo de agua y menor índice de desarrollo - Bajo riesgo de escasez de agua
    * Alcaldías con menor consumo de agua y mayor índice de desarrollo - Medio riesgo de escasez de agua

* Principal fuente de consumo de agua (doméstico, no doméstico, mixto)
* Colonias con un consumo desproporcionado en relación con el número de inmuebles - Posible riesgo de escasez de agua
* Alcaldías con un consumo desproporcionado en relación con el número de inmuebles - Posible riesgo de escasez de agua

**Accionables**    
* Alcaldías/Colonias con mayor consumo y con un índice de desarrollo bajo (posible riesgo de escasez de agua)
* Colonias con un consumo desproporcionado en relación con el número de inmuebles (posible riesgo de escasez de agua)

#### Data Processing

* Cleaning, feature engineering, and formatting

In [ ]:
data_consumo_19[['fecha_referencia', 'anio', 'bimestre', 'indice_des', 'colonia', 'alcaldia', 'latitud', 'longitud']].drop_duplicates(inplace=True)

In [ ]:
# Date 
data_consumo_19['fecha_referencia'] = pd.to_datetime(data_consumo_19['fecha_referencia'], format='ISO8601')

# Adding derived columns for total consumption
data_consumo_19['inmuebles_domesticos'] = data_consumo_19['consumo_total_dom'] / data_consumo_19['consumo_prom_dom']
data_consumo_19['inmuebles_no_domesticos'] = data_consumo_19['consumo_total_no_dom'] / data_consumo_19['consumo_prom_no_dom']
data_consumo_19['inmuebles_mixtos'] = data_consumo_19['consumo_total_mixto'] / data_consumo_19['consumo_prom_mixto']

# replace NaN values with 0 in the derived column
data_consumo_19['inmuebles_domesticos'].fillna(0, inplace=True)
data_consumo_19['inmuebles_no_domesticos'].fillna(0, inplace=True)
data_consumo_19['inmuebles_mixtos'].fillna(0, inplace=True)

# Data structure and column ordering
data_consumo_19 = data_consumo_19[['fecha_referencia', 'consumo_total', 'inmuebles_domesticos', 'consumo_total_dom', 'consumo_prom_dom', 'inmuebles_no_domesticos', 'consumo_total_no_dom', 'consumo_prom_no_dom', 'inmuebles_mixtos', 'consumo_total_mixto', 'consumo_prom_mixto', 'indice_des', 'colonia', 'alcaldia']]

# Adding total consumption and total number of properties
data_consumo_19['total_inmuebles'] = data_consumo_19['inmuebles_domesticos'] + data_consumo_19['inmuebles_no_domesticos'] + data_consumo_19['inmuebles_mixtos']


# Important : Filter the data to only include the last date of 2019
print("Dates in the dataset:", data_consumo_19['fecha_referencia'].unique())
data_consumo_19_f = data_consumo_19[data_consumo_19['fecha_referencia'] == '2019-06-30']

data_consumo_19_f.head()

#### First Aggregation

In [ ]:
t = data_consumo_19_f[['indice_des', 'colonia',	'alcaldia', 'fecha_referencia']]
t[t.duplicated()]

In [ ]:
# Neighborhood aggregation (since we have duplicated neighborhoods, we can aggregate the data by summing the total consumption)
data_consumo_19_agg = data_consumo_19_f.groupby(['colonia', 'alcaldia']).agg({
    'consumo_total': 'sum',
    'inmuebles_domesticos': 'sum',
    'consumo_total_dom': 'sum',
    #'consumo_prom_dom': 'mean',
    'inmuebles_no_domesticos': 'sum',
    'consumo_total_no_dom': 'sum',
    #'consumo_prom_no_dom': 'mean',
    'inmuebles_mixtos': 'sum',
    'consumo_total_mixto': 'sum',
    'total_inmuebles': 'sum',
    #'indice_des': 'mean'
}).reset_index()


# We filter out rows where all consumption values are zero
data_consumo_19_agg = data_consumo_19_agg[~(data_consumo_19_agg.iloc[:, 2:] == 0).all(axis=1)]

# insert a pivot and create new columns based on indice_des column using the values
share_idx_des = data_consumo_19_f.pivot_table(
    index=['colonia', 'alcaldia'],
    columns='indice_des',
    values='total_inmuebles',
    aggfunc='sum'
).reset_index()

all_agg = data_consumo_19_agg.merge(share_idx_des, on=['colonia', 'alcaldia'], how='left')
all_agg.fillna(0, inplace=True)

all_agg.head()

**Principal**:

* ¿Cuáles son las colonias con mayor y menor consumo de agua? - Usar total agregando por colonia y alcaldía
    * Sin filtros
    * Filtrando por colonias con inmuebles domésticos representando más del 50% del consumo total de agua
    * Filtrando por colonias con inmuebles no domésticos representando más del 50% del consumo total de agua
    * Filtrando por colonias con inmuebles domésticos representando más del 50% del número de inmuebles, pero inmuebles no domésticos representando más del 50% del consumo total de agua - Colonias donde comprar una casa puede representar una posible escasez a futuro

* Sin Filtros

In [ ]:
nf_h = all_agg.sort_values(by='consumo_total', ascending=False)
# nf_h.head(20)

In [ ]:
d = nf_h.head(20).copy()

labels = [
    f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
    for _, row in d.iterrows()
]
sizes = d['consumo_total']

plt.figure(figsize=(16, 9))
squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 11, 'color': 'black'})
plt.title("Top 20 Colonias con Mayor Consumo Total de Agua (m³)")
plt.axis('off')
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción del Índice de Desarrollo Urbano (IDU) de las Top 20 colonias más consumidoras")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['inmuebles_domesticos'].sum(), d['inmuebles_no_domesticos'].sum(), d['inmuebles_mixtos'].sum()]
labels = ['inmuebles_domesticos', 'inmuebles_no_domesticos', 'inmuebles_mixtos']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción del tipo de inmuebles de las Top 20 colonias más consumidoras")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
nf_t = all_agg.sort_values(by='consumo_total', ascending=True)

In [ ]:
d = nf_t.head(20).copy()

labels = [
    f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
    for _, row in d.iterrows()
]
sizes = d['consumo_total']

plt.figure(figsize=(16, 9))
squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 8, 'color': 'black'})
plt.title("Top 10 Colonias con Menor Consumo Total de Agua (m³)")
plt.axis('off')
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Top 20 menos consumidoras")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
yf_dom_h = all_agg[all_agg.iloc[:,2:]['consumo_total_dom'] / all_agg.iloc[:,2:]['consumo_total'] > 0.5].sort_values(by='consumo_total', ascending=False)

In [ ]:
d = yf_dom_h.head(20).copy()

labels = [
    f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
    for _, row in d.iterrows()
]
sizes = d['consumo_total']

plt.figure(figsize=(16, 9))
squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 11, 'color': 'black'})
plt.title("Top 10 Colonias con Inmuebles Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('off')
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Inmuebles Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

* Filtrando por alcaldías con inmuebles no domésticos representando más del 50% del consumo total de agua

In [ ]:
yf_no_dom_h = all_agg[all_agg.iloc[:,2:]['consumo_total_no_dom'] / all_agg.iloc[:,2:]['consumo_total'] > 0.5].sort_values(by='consumo_total', ascending=False)

In [ ]:
d = yf_no_dom_h.head(20).copy()

labels = [
    f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
    for _, row in d.iterrows()
]
sizes = d['consumo_total']

plt.figure(figsize=(16, 9))
squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 11, 'color': 'black'})
plt.title("Top 10 Colonias con Inmuebles No Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('off')
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Inmuebles No Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

* Filtrando por alcaldías con inmuebles domésticos representando más del 50% del número de inmuebles, pero inmuebles no domésticos representando más del 50% del consumo total de agua

In [ ]:
yf_dom_no_h = all_agg[(all_agg.iloc[:,2:]['inmuebles_domesticos'] / all_agg.iloc[:,2:]['total_inmuebles'] > 0.8) & 
                    (all_agg.iloc[:,2:]['consumo_total_no_dom'] / all_agg.iloc[:,2:]['consumo_total'] > 0.6)].sort_values(by='consumo_total', ascending=False)

In [ ]:
d = yf_dom_no_h.head(20).copy()

labels = [
    f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
    for _, row in d.iterrows()
]
sizes = d['consumo_total']

plt.figure(figsize=(16, 9))
squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 8, 'color': 'black'})
plt.title("Top 10 Colonias con más del 50% Inmuebles Dom e Inmuebles No Dom que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('off')
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Top 10 con más del 50% Inmuebles Dom e Inmuebles No Dom que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

* Existe una relación entre el índice de desarrollo y el consumo de agua en las colonias, donde las colonias con un índice de desarrollo más bajo tienden a tener un mayor consumo de agua or menor consumo de agua, y viceversa. 

In [ ]:
# Correlación entre tipo de inmueble y consumo doméstico
correlation_matrix = all_agg[['ALTO', 'BAJO', 'MEDIO', 'POPULAR', 'consumo_total_dom']].corr()

# Heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Matriz de Correlación")
plt.show()

In [ ]:
all_agg['mayoria_idx'] = all_agg[['ALTO', 'BAJO', 'MEDIO', 'POPULAR']].idxmax(axis=1)
dict1 = {
    'ALTO': 'blue',
    'MEDIO': 'green',
    'BAJO': 'orange',
    'POPULAR': 'red'
    }

all_agg['mayoria_type'] = all_agg[['inmuebles_domesticos', 'inmuebles_no_domesticos', 'inmuebles_mixtos']].idxmax(axis=1)

dict2 = {'inmuebles_domesticos' : 'blue',
         'inmuebles_no_domesticos' : 'green',
         'inmuebles_mixtos' : 'orange'}

all_agg['consumo_mayoria'] =  all_agg[['consumo_total_dom', 'consumo_total_no_dom', 'consumo_total_mixto']].idxmax(axis=1)
dict3 = {'consumo_total_dom' : 'blue',
         'consumo_total_no_dom' : 'green',
         'consumo_total_mixto' : 'orange'}

In [ ]:
plt.figure(figsize=(10, 6))

for category, color in dict1.items():
    subset = all_agg[all_agg['mayoria_idx'] == category]
    plt.scatter(subset['total_inmuebles'], subset['consumo_total'],
                alpha=0.7, c=color, label=f'Índice de Desarrollo {category}')


plt.xlabel('Total de Inmuebles')
plt.ylabel('Consumo Total de Agua (m³)')
plt.title('Relación entre Total de Inmuebles y Consumo Total de Agua por Colonia')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(30, 7))  # Increase figsize for better spacing

category = 'ALTO'
subset_alto = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
axes[0].scatter(subset_alto['total_inmuebles'], subset_alto['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[0].set_title('Total Inmuebles Desarrollo Alto vs Consumo agua (m³)')
axes[0].set_ylim(0, 130000)

category = 'MEDIO'
subset_medio = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
axes[1].scatter(subset_medio['total_inmuebles'], subset_medio['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[1].set_title('Total Inmuebles Desarrollo Medio vs Consumo agua (m³)')
axes[1].set_ylim(0, 130000)

category = 'BAJO'
subset_bajo = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
axes[2].scatter(subset_bajo['total_inmuebles'], subset_bajo['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[2].set_title('Total Inmuebles Desarrollo Bajo vs Consumo agua (m³)')
axes[2].set_ylim(0, 130000)

category = 'POPULAR'
subset_pop = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
axes[3].scatter(subset_pop['total_inmuebles'], subset_pop['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[3].set_title('Total Inmuebles Desarrollo Popular vs Consumo agua (m³)')
axes[3].set_ylim(0, 130000)

plt.show()

In [ ]:
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['total_inmuebles'].sum()}")
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['total_inmuebles'].sum()}")
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['total_inmuebles'].sum()}")
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['total_inmuebles'].sum()}")

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(30, 7))  # Increase figsize for better spacing

category = 'ALTO'
subset_alto = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_domesticos'].sum()
axes[0].scatter(subset_alto['inmuebles_domesticos'], subset_alto['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[0].set_title(f'Consumo vs # Inmuebles Dom con Alto IDU' + f' ({int(total_cat)} inmuebles)')
axes[0].set_ylim(0, 100000)
axes[0].set_xlabel('Total Inmuebles')
axes[0].set_ylabel('Consumo de Agua (m³)')

category = 'MEDIO'
subset_medio = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_domesticos'].sum()
axes[1].scatter(subset_medio['inmuebles_domesticos'], subset_medio['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[1].set_title(f'Consumo vs # Inmuebles Dom con Medio IDU' + f' ({int(total_cat)} inmuebles)')
axes[1].set_ylim(0, 100000)
axes[1].set_xlabel('Total Inmuebles')
axes[1].set_ylabel('Consumo de Agua (m³)')

category = 'BAJO'
subset_bajo = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_domesticos'].sum()
axes[2].scatter(subset_bajo['inmuebles_domesticos'], subset_bajo['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[2].set_title(f'Consumo vs # Inmuebles Dom con Bajo IDU' + f' ({int(total_cat)} inmuebles)')
axes[2].set_ylim(0, 100000)
axes[2].set_xlabel('Total Inmuebles')
axes[2].set_ylabel('Consumo de Agua (m³)')

category = 'POPULAR'
subset_pop = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_domesticos'].sum()
axes[3].scatter(subset_pop['inmuebles_domesticos'], subset_pop['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[3].set_title(f'Consumo vs # Inmuebles Dom con Popular IDU' + f' ({int(total_cat)} inmuebles)')
axes[3].set_ylim(0, 100000)
axes[3].set_xlabel('Total Inmuebles')
axes[3].set_ylabel('Consumo de Agua (m³)')

plt.show()

In [ ]:
print(f"El consumo promedio de casas con un índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['consumo_total_dom'].sum()}")
print()

print(f"El consumo promedio de casas con un índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['consumo_total_dom'].sum()}")
print()

print(f"El consumo promedio de casas con un índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['consumo_total_dom'].sum()}")
print()

print(f"El consumo promedio de casas con un índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['consumo_total_dom'].sum()}")
print()

In [ ]:
data_consumo_19_f[data_consumo_19_f['consumo_total_dom'] > 40000].sort_values(by='consumo_total_dom', ascending=False)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(30, 7))  # Increase figsize for better spacing

category = 'ALTO'
subset_alto = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_no_domesticos'].sum()
axes[0].scatter(subset_alto['inmuebles_no_domesticos'], subset_alto['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[0].set_title(f'Consumo vs # Inmuebles No Dom con Alto IDU' + f' ({int(total_cat)} inmuebles)')
axes[0].set_ylim(0, 100000)
axes[0].set_xlabel('Total Inmuebles')
axes[0].set_ylabel('Consumo de Agua (m³)')

category = 'MEDIO'
subset_medio = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_no_domesticos'].sum()
axes[1].scatter(subset_medio['inmuebles_no_domesticos'], subset_medio['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[1].set_title(f'Consumo vs # Inmuebles No Dom con Medio IDU' + f' ({int(total_cat)} inmuebles)')
axes[1].set_ylim(0, 100000)
axes[1].set_xlabel('Total Inmuebles')
axes[1].set_ylabel('Consumo de Agua (m³)')

category = 'BAJO'
subset_bajo = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_no_domesticos'].sum()
axes[2].scatter(subset_bajo['inmuebles_no_domesticos'], subset_bajo['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[2].set_title(f'Consumo vs # Inmuebles No Dom con Bajo IDU' + f' ({int(total_cat)} inmuebles)')
axes[2].set_ylim(0, 100000)
axes[2].set_xlabel('Total Inmuebles')
axes[2].set_ylabel('Consumo de Agua (m³)')

category = 'POPULAR'
subset_pop = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_no_domesticos'].sum()
axes[3].scatter(subset_pop['inmuebles_no_domesticos'], subset_pop['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
axes[3].set_title(f'Consumo vs # Inmuebles No Dom con Popular IDU' + f' ({int(total_cat)} inmuebles)')
axes[3].set_ylim(0, 100000)
axes[3].set_xlabel('Total Inmuebles')
axes[3].set_ylabel('Consumo de Agua (m³)')

plt.show()

In [ ]:
data_consumo_19_f[data_consumo_19_f['consumo_total_no_dom'] > 40000].sort_values(by='consumo_total_no_dom', ascending=False)

* ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))

sizes = [data_consumo_19_f['consumo_total_dom'].sum(), data_consumo_19_f['consumo_total_no_dom'].sum(), data_consumo_19_f['consumo_total_mixto'].sum()]
labels = ['consumo_total_dom', 'consumo_total_no_dom', 'consumo_total_mixto']

plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de la Fuente del Consumo")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

* ¿Hay colonias con un consumo desproporcionado en relación con el número de inmuebles? ¿Cuáles? - Usar total agregando por colonia y alcaldía

In [ ]:
# Podemos encontrar ese consumo desproporcionado utilizando el consumo promedio
consumo_prom = data_consumo_19_f[['consumo_prom_dom', 'consumo_prom_no_dom', 'consumo_prom_mixto', 'indice_des', 'colonia', 'alcaldia']]
consumo_prom.sort_values(by='consumo_prom_dom', ascending=False).head(20)

* Cuántas colonias representan más del 50% del consumo total doméstico

In [ ]:
suma = 0
i = 0
while suma < all_agg['consumo_total_dom'].sum() * 0.5: 
    suma += all_agg.sort_values(by='consumo_total_dom', ascending=False)['consumo_total_dom'][i]
    i +=1
    
i

* Alcaldías que más gastan

In [ ]:
# Neighborhood aggregation (since we have duplicated neighborhoods, we can aggregate the data by summing the total consumption)
data_consumo_19_agg_alc = data_consumo_19_f.groupby(['alcaldia']).agg({
    'consumo_total': 'sum',
    'inmuebles_domesticos': 'sum',
    'consumo_total_dom': 'sum',
    #'consumo_prom_dom': 'mean',
    'inmuebles_no_domesticos': 'sum',
    'consumo_total_no_dom': 'sum',
    #'consumo_prom_no_dom': 'mean',
    'inmuebles_mixtos': 'sum',
    'consumo_total_mixto': 'sum',
    'total_inmuebles': 'sum',
    #'indice_des': 'mean'
}).reset_index()


# We filter out rows where all consumption values are zero
data_consumo_19_agg_alc = data_consumo_19_agg_alc[~(data_consumo_19_agg_alc.iloc[:, 2:] == 0).all(axis=1)]

# insert a pivot and create new columns based on indice_des column using the values
share_idx_des_alc = data_consumo_19_f.pivot_table(
    index=['alcaldia'],
    columns='indice_des',
    values='total_inmuebles',
    aggfunc='sum'
).reset_index()

all_agg_alc = data_consumo_19_agg_alc.merge(share_idx_des_alc, on=['alcaldia'], how='left')
all_agg_alc.fillna(0, inplace=True)

all_agg_alc.sort_values(by='consumo_total', ascending=False)

In [ ]:
all_agg_alc.sort_values(by='consumo_total_dom', ascending=False)

In [ ]:
all_agg_alc.sort_values(by='consumo_total_no_dom', ascending=False)

* ¿Cuál es el la proporción del índice de desarrollo en dichas alcaldías?

In [ ]:
# En toda la ciudad
plt.figure(figsize=(150,10))
sizes = [all_agg_alc['ALTO'].sum(), all_agg_alc['MEDIO'].sum(), all_agg_alc['BAJO'].sum(), all_agg_alc['POPULAR'].sum()]
labels = ['ALTO', 'MEDIO', 'BAJO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=0, colors=sns.color_palette("pastel", len(sizes)))
plt.title('Proporción de, Índice de Desarrollo en la Ciudad de México de todo tipo de inmuebles')
plt.show()

* ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?

In [ ]:
plt.figure(figsize=(15,10))
sizes = [all_agg_alc['consumo_total_dom'].sum(), all_agg_alc['consumo_total_no_dom'].sum(), all_agg_alc['consumo_total_mixto'].sum()]
labels = ['Consumo Doméstico', 'Consumo No Doméstico', 'Consumo Mixto']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=0, colors=sns.color_palette('pastel', len(sizes)))
plt.title('Proporción de la Fuente de Consumo de agua en CDMX')
plt.show()

* Alcaldías con un consumo desproporcionado

In [ ]:
all_agg_alc['consumo_dom_prom'] = all_agg_alc['consumo_total_dom'] / all_agg_alc['inmuebles_domesticos']
all_agg_alc['consumo_no_dom_prom'] = all_agg_alc['consumo_total_no_dom'] / all_agg_alc['inmuebles_no_domesticos']
all_agg_alc['consumo_mixto_prom'] = all_agg_alc['consumo_total_mixto'] / all_agg_alc['inmuebles_mixtos']

In [ ]:
all_agg_alc.sort_values(by='consumo_dom_prom', ascending=False)

### Factibilidad Hídrica en Ciudad de México (2016)


| Grado de Factibilidad Hídrica | Descripción / Significado                                | Color       
| ----------------------------- | -------------------------------------------------------- | ----------- 
| Alta factibilidad 🌟          | Zonas con alta capacidad para mitigar riesgo de escasez  | 🟩 Verde    
| Media-alta factibilidad ⚠️    | Zonas con buena capacidad, pero con algunas limitaciones | 🟨 Amarillo 
| Media-baja factibilidad 🔶    | Zonas con capacidad limitada para mitigar riesgo         | 🟧 Naranja  
| Baja factibilidad ❌           | Zonas con poca o nula capacidad para mitigar riesgo      | 🟥 Rojo    

In [ ]:
dir_color = {'ROJO' : 'red',
             'AMARILLO' : 'yellow',
             'NARANJA' : 'orange',
             'VERDE' : 'green'}

# Map the colors to a new column
data_feasibility['color'] = data_feasibility['fact_hidr'].map(dir_color)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
data_feasibility.plot(ax=ax, color=data_feasibility['color'], edgecolor='black', linewidth=0.1)
plt.title('Factibilidad Hídrica en Ciudad de México', fontsize=14)
plt.axis('off')
plt.show()

In [ ]:
all_agg[all_agg['colonia'] == 'SAN ALVARO']

In [ ]:
feasi_and_consumo = data_feasibility.merge(all_agg, left_on=['NOMBRE'], right_on=['colonia'])
feasi_and_consumo.head()

### Consumo Habitacional (2019)

Diccionario : 

**SUM_cons_t**   
Suma del consumo total de agua por colonia (m3) por bimestre

**MEAN_cons_**   
Promedio de consumo total de colonia (m3) por bimestre

**VIV2010**   
Vivienda censo 2010

**PROMVIVCON**   
"Promedio de consumo total de agua de la colonia por número de viviendas"

**C_PROMVIVC**   
"Campo reclasificado en cuantiles en 5 rangos a partir de "PROMVIVCON"; donde el valor "5" corresponde a un consumo de agua muy alto, mientras que el número "1" a un consumo bajo"



In [ ]:
# Bulding a classification depending on total consumption of each county
all_agg['q_class_all'] = pd.qcut(all_agg['consumo_total'], q=5, labels=False)
all_agg['q_class_all'] = all_agg['q_class_all'] + 1

all_agg['q_class_dom'] = pd.qcut(all_agg['consumo_total_dom'], q=5, labels=False)
all_agg['q_class_dom'] = all_agg['q_class_dom'] + 1

# mergin two dfs
data_hab = hab_consoumption[['geometry', 'alcaldia', 'colonia', 'PROMVIVCON', 'C_PROMVIVC', 'SUM_cons_t']].merge(all_agg, left_on=['alcaldia','colonia'], right_on=['alcaldia', 'colonia'], how='left')
# data_hab.loc[data_hab['q_class_dom'].isna(), 'q_class_dom'] = data_hab.loc[data_hab['q_class_dom'].isna(), 'C_PROMVIVC']

data_hab.loc[data_hab['q_class_all'].isna(), 'q_class_all'] = data_hab.loc[data_hab['q_class_all'].isna(), 'C_PROMVIVC']
data_hab.loc[data_hab['consumo_total'].isna(), 'consumo_total'] = data_hab.loc[data_hab['consumo_total'].isna(), 'SUM_cons_t']

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10, 10))
data_hab.plot(column='q_class_all', legend=True, edgecolor='black', linewidth=0.1, ax=ax, cmap='viridis')
plt.title('Consumo de Agua en Ciudad de México', fontsize=14)
plt.axis('off')
plt.show()

In [ ]:
data_hab_agg = data_hab.groupby('alcaldia').agg({
    'consumo_total' : 'sum',
    'consumo_total_dom' : 'sum',
    'consumo_total_no_dom' : 'sum',
     'consumo_total_mixto' : 'sum',
}).reset_index()
data_hab_agg.sort_values(by='consumo_total', inplace=True)

# Set positions for the bars
alcaldias = data_hab_agg['alcaldia']
dom = data_hab_agg['consumo_total_dom']
no_dom = data_hab_agg['consumo_total_no_dom']
mixto = data_hab_agg['consumo_total_mixto']

# Create stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(alcaldias, dom, label='Doméstico')
ax.barh(alcaldias, no_dom, left=dom, label='No Doméstico')
ax.barh(alcaldias, mixto, left=dom + no_dom, label='Mixto')

# Labels and legend
ax.set_ylabel('Consumo Total')
ax.set_title('Consumo de Agua por Alcaldía y Tipo de Uso')
ax.legend()

# plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Probabilidad de Escasez

In [ ]:

import requests
import json
import re
import plotly.express as px

# Load Mexico Map and states
# url = "https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json"
# geojson = requests.get(url).json()

# Load CDMX alcaldías GeoJSON
with open('data/water/sequia/limite-de-las-alcaldas.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
alcaldias_cdmx = gpd.GeoDataFrame.from_features(data['features'])
alcaldias_cdmx = alcaldias_cdmx.set_crs(epsg=4326, inplace=True)
geojson = alcaldias_cdmx.__geo_interface__

# Step 1: Identify id_vars (the static columns)
id_vars = ['Region de lluvias y fecha de solicitud de corroboración', 'CVE_CONCATENADA', 'CVE_ENT', 'CVE_MUN', 'NOMBRE_MUN', 'ENTIDAD',
           'ORG_CUENCA', 'CLV_OC', 'CON_CUENCA', 'CVE_CONC']

# Step 2: Melt all other columns
df_melted = data_drought.melt(id_vars=id_vars, var_name='variable', value_name='value')

# Create category and date columns
category = []
date = []
for i in df_melted['variable'].astype(str):
    tup = i.split('/')
    category.append(tup[0])
    date.append(tup[1])
    
df_melted['CATEGORY'] = category
df_melted['CATEGORY'] = df_melted['CATEGORY'].str.replace(r'\.\d+$', '', regex=True)

df_melted['DATE'] = date
df_melted['DATE'] = pd.to_datetime(df_melted['DATE'])
df_melted['MONTH'] = df_melted['DATE'].dt.month_name()
df_melted['YEAR'] = df_melted['DATE'].dt.year
df_melted['DATE'] = df_melted['DATE'].dt.strftime('%Y-%m')

# Mexico city filter, adding map and taking MAGNITUD value onl7
mxc_drought = df_melted[(df_melted['ENTIDAD'] == 'Ciudad de México') & (df_melted['CATEGORY'] == 'MAGNITUD')]
mxc_drought = pd.merge(mxc_drought, alcaldias_cdmx[['CVE_MUN', 'geometry']], on='CVE_MUN', how='left')

color_map = {
    'SIN SEQUIA': '#f0f0f0',     # Very light gray / near white
    'PRE-ALERTA': '#1a9850',     # Light gray
    'VERDE': '#a6d96a',          # Soft green
    'AMARILLO': '#ffffbf',       # Yellow
    'NARANJA': '#fdae61',        # Orange
    'ROJO': '#d73027'            # Red
}

mxc_drought['color'] = mxc_drought[mxc_drought['CATEGORY'] == 'MAGNITUD']['value'].map(color_map)
mxc_drought = gpd.GeoDataFrame(mxc_drought, geometry='geometry')
# mxc_drought["geometry"] = mxc_drought["geometry"].simplify(tolerance=0.0005, preserve_topology=True)

# Data Processing
number_map = {
    'SIN SEQUIA': 1,     # Very light gray / near white
    'PRE-ALERTA': 2,     # Light gray
    'VERDE': 3,          # Soft green
    'AMARILLO': 4,       # Yellow
    'NARANJA': 5,        # Orange
    'ROJO': 6            # Red
}

mxc_drought['VALUE'] = mxc_drought[mxc_drought['CATEGORY'] == 'MAGNITUD']['value'].map(number_map)
mxc_drought.head()

* Comparison between all 2003, 2013 and 2023 years (Animated plots)

In [ ]:
# mxc_drought['DATE'] = pd.to_datetime(mxc_drought['DATE'])

# mxc_drought_2003_s = mxc_drought[mxc_drought['DATE'] == '2003-01-01']
# mxc_drought_2013_s = mxc_drought[mxc_drought['DATE'] == '2013-01-01']
# # mxc_drought_2023_s = mxc_drought[mxc_drought['DATE'] == '2023-01-01']

# mxc_drought_2003_a = mxc_drought[(mxc_drought['DATE'] >= '2003-01-01') & (mxc_drought['DATE'] <= '2003-12-01')]
# mxc_drought_2013_a = mxc_drought[(mxc_drought['DATE'] >= '2013-01-01') & (mxc_drought['DATE'] <= '2013-12-01')]
# mxc_drought_2023_a = mxc_drought[(mxc_drought['DATE'] >= '2023-01-01') & (mxc_drought['DATE'] <= '2023-12-01')]

In [ ]:
mxc_drought['VALUE'] = mxc_drought['VALUE'].astype(int)

In [ ]:
def plot_static_map(df, title, show=True, write=False, file_name=None) : 
    # Define your color mapping
    color_discrete_map = {
        'SIN SEQUIA': '#f0f0f0',
        'PRE-ALERTA': '#d9d9d9',
        'VERDE': '#a6d96a',
        'AMARILLO': '#ffffbf',
        'NARANJA': '#fdae61',
        'ROJO': '#d73027'
    }

    fig = px.choropleth_mapbox(
        df,
        geojson=df.__geo_interface__,
        locations=df.index,  # You can use index if each row is unique
        color="value",  # Use categorical value, not raw hex codes
        hover_name="NOMBRE_MUN",
        hover_data=["CATEGORY", "value", 'DATE'],
        mapbox_style="carto-positron",
        zoom=9,
        center={"lat": 19.33, "lon": -99.13},
        opacity=0.8,
        color_discrete_map=color_discrete_map
    )


    fig.update_layout(
        margin={"r":0, "t":30, "l":0, "b":0},
        title=title,
        width=1000,    # width in pixels
        height=700,    # height in pixels
        showlegend=False
    )

    if show:
        # Show hover labels directly on the map
        fig.update_traces()
        fig.show()

    if write : 
        if file_name == None : 
            file_name = 'fig.html'
        fig.write_html(file_name)
    
def plot_animated_map(df, title, show=True, write=False, file_name=None) : 
    # Convert DATE to string if it's datetime (required by Plotly animation)
    # df["DATE"] = df["DATE"].astype(str)

    # Define category → color mapping
    color_discrete_map = {
        'SIN SEQUIA': '#f0f0f0',
        'PRE-ALERTA': '#d9d9d9',
        'VERDE': '#a6d96a',
        'AMARILLO': '#ffffbf',
        'NARANJA': '#fdae61',
        'ROJO': '#d73027'
    }
    
    color_map = {
    "1": '#f0f0f0',     # Very light gray / near white
    "2": '#1a9850',     # Light gray
    "3": '#a6d96a',          # Soft green
    "4": '#ffe066',       # Yellow
    "5": '#fdae61',        # Orange
    "6": '#d73027'            # Red
    }
    
    mxc_drought_t = df[['DATE', 'geometry', 'NOMBRE_MUN', 'VALUE', 'color']]
    
    vmin = np.log10(mxc_drought_t["VALUE"].min())
    vmax = np.log10(mxc_drought_t["VALUE"].max())

    def normalize(val):
        return (val - 1) / (6 - 1)

    custom_scale = [
        (normalize(1), "#f0f0f0"),
        (normalize(2), "#1a9850"),
        (normalize(3), "#a6d96a"),
        (normalize(4), "#ffe066"),
        (normalize(5), "#fdae61"),
        (normalize(6), "#d73027"),
    ]    

    fig = px.choropleth_mapbox(mxc_drought_t,
                        geojson=mxc_drought_t.__geo_interface__,
                        locations="NOMBRE_MUN",
                        color=np.log10(mxc_drought_t["VALUE"]),
                        hover_name="NOMBRE_MUN",
                        hover_data=["VALUE"],
                        animation_frame='DATE',
                        featureidkey='properties.NOMBRE_MUN',  # Adjust based on GeoJSON field
                        # color_continuous_midpoint = 1,
                        color_continuous_scale=custom_scale,
                        range_color=[0,1],
                        # color_discrete_map=color_map,
                        mapbox_style="carto-positron",
                        center={"lat": 19.33, "lon": -99.1332},  # Center on Mexico City
                        zoom=9,)

    fig.update_layout(margin=dict(l=20,r=0,b=0,t=70,pad=0),
                    paper_bgcolor="white",
                    width = 1000,
                    height= 700,
                    title_text = 'Evolución de la Sequía en la Ciudad de México 2003-2023',
                    font_size=18,
                    )
    # fig.show()

    if show : 
        # Show hover labels directly on the map
        # fig.update_traces()
        fig.show()
        
    if write : 
        if file_name == None:
            file_name = 'fig.html'
        fig.write_html(file_name)

In [ ]:
# plot_static_map(mxc_drought_2003_s, 'Sequía en Ciudad de México en Oct 2003')
# plot_animated_map(mxc_drought_2003_a, 'Sequía en Ciudad de México en el año 2003', show=False, write=False, file_name='sequia_mxc_2003.html')

In [ ]:
# plot_static_map(mxc_drought_2013_s, 'Sequía en Ciudad de México en Oct 2013')
# plot_animated_map(mxc_drought_2013_a, 'Sequía en Ciudad de México en el año 2013', show=False, write=True, file_name='sequia_mxc_2013.html')

In [ ]:
# plot_static_map(mxc_drought_2023_s, 'Sequía en Ciudad de México en Oct 2023')
# plot_animated_map(mxc_drought_2023_a, 'Sequía en Ciudad de México en el año 2023', show=False, write=True, file_name='sequia_mxc_2023.html')

* Comparison between each year

In [ ]:
# Filtering by year
# mxc_drought_years_only = mxc_drought[(mxc_drought['MONTH'] == 'October')]
# plot_animated_map(mxc_drought, 'Evolución de la Sequía en la Ciudad de México 2003-2023 (Oct only)', show=False, write=True, file_name='sequia_evolucion_mxc_oct.html')

#### Insights

- Entender cuánto ha cambiado con el tiempo el número de alcaldías en alerta con sequía
    * Maybe del 2023 vs T-5 años
    * Maybe del 2023 vs T-10 años
    * Maybe del 2023 vs T-20 años

Cuantitativo y en gráfica

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2023]['value'].value_counts()

In [ ]:
avg_status_per_year = (mxc_drought[['YEAR', 'CVE_MUN', 'NOMBRE_MUN', 'VALUE']].groupby(by=['YEAR', 'CVE_MUN','NOMBRE_MUN'])['VALUE'].sum()/12).reset_index()
avg_merge = pd.merge(avg_status_per_year, alcaldias_cdmx[['geometry', 'CVE_MUN']], on='CVE_MUN', how='left')
avg_merge = gpd.GeoDataFrame(avg_merge)

# avg_merge['VALUE'] = avg_merge['VALUE']/10

In [ ]:
def normalize(val):
    return (val - 1) / (6 - 1)

custom_scale = [
        (normalize(1), "#f0f0f0"),
        (normalize(2), "#1a9850"),
        (normalize(3), "#a6d96a"),
        (normalize(4), "#ffe066"),
        (normalize(5), "#fdae61"),
        (normalize(6), "#d73027"),
    ]    

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2023],
#     geojson=avg_merge[avg_merge['YEAR'] == 2023].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2023].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2023]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2023',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

# file_name = 'sequia_promedio_2023.html'
# fig.write_html(file_name)

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge,
#     geojson=avg_merge.__geo_interface__,
#     locations="NOMBRE_MUN",  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     animation_frame='YEAR',
#     featureidkey='properties.NOMBRE_MUN',  # Adjust based on GeoJSON field
#     animation_group="NOMBRE_MUN",
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2003-2023',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

# file_name = 'sequia_promedio_evolution_3.html'
# fig.write_html(file_name)

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2018]['value'].value_counts()

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2018],
#     geojson=avg_merge[avg_merge['YEAR'] == 2018].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2018].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2018]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2018',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

# file_name = 'sequia_promedio_2023.html'
# fig.write_html(file_name)

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2013]['value'].value_counts()

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2013],
#     geojson=avg_merge[avg_merge['YEAR'] == 2013].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2013].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2013]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2013',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2003]['value'].value_counts()

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2003],
#     geojson=avg_merge[avg_merge['YEAR'] == 2003].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2003].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2003]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2003',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

* All history 2003-2023

In [ ]:
# # Aggregate data
# t = mxc_drought.groupby(by=['DATE', 'MONTH', 'YEAR'])['VALUE'].mean().reset_index()
# t['DATE'] = pd.to_datetime(t['DATE'])

# # Filter date range (2003 to 2023)
# t_filtered = t[(t['DATE'] >= "2003-01-01") & (t['DATE'] <= "2023-12-31")]

# # Create Plotly line plot
# fig = px.line(
#     t_filtered,
#     x='DATE',
#     y='VALUE',
#     markers=True,
#     labels={'VALUE': 'Drought Category', 'DATE': 'Date'},
#     title='Escasez del agua en Ciudad de México (0 - sin escasez, 6 - alta escasez)'
# )

# # Set y-axis ticks manually
# fig.update_yaxes(tickmode='array', tickvals=[1, 2, 3, 4, 5, 6])

# # Format x-axis range and tick labels
# fig.update_xaxes(
#     range=[t_filtered['DATE'].min(), t_filtered['DATE'].max()],
#     tickformat="%Y",
#     dtick="M12",  # One tick every 12 months
#     tickangle=45
# )

# fig.update_layout(
#     width=2400,
#     height=650,
#     template='plotly_white'
# )

# fig.show()


### Reportes de Agua

Info : 
- Data ranging from 2018 to 2024

* History of reports
    * The most common report
    * Separate all this by alcaldia and colonia

* Analyze time and identify trends/patterns in which all this reports are made.
    * Tipically after what hour or during which range of hours.

In [ ]:
data_reports_22_24 = data_water_reps_22_24[['fecha_reporte','hora_reporte', 'reporte', 'alcaldia_catalogo', 'colonia_catalogo', 'longitud', 'latitud']]
data_reports_hist_22 = data_water_reps_hist[['fecha','tipo_de_falla', 'alcaldia', 'colonia_datos_abiertos', 'longitud', 'latitud']]

new_names_cols = {'tipo_de_falla' : 'reporte', 
                  'alcaldia_catalogo' : 'alcaldia', 
                  'colonia_catalogo' : 'colonia',
                  'colonia_datos_abiertos' : 'colonia',
                  'fecha_reporte' : 'fecha'
                  }

data_reports_hist_22.rename(columns=new_names_cols, inplace=True)
data_reports_22_24.rename(columns=new_names_cols, inplace=True)

all_reports = pd.concat([data_reports_22_24, data_reports_hist_22], axis=0)

all_reports['fecha'] = pd.to_datetime(all_reports['fecha'])
all_reports['year'] = all_reports['fecha'].dt.year
all_reports['month_name'] = all_reports['fecha'].dt.month_name()
all_reports['month'] = all_reports['fecha'].dt.month

all_reports = all_reports.dropna(subset=['alcaldia', 'colonia'])
all_reports.head()

In [ ]:
# Most common reports per year
reports_count_p_y = all_reports.groupby(by=['year', 'reporte']).size().reset_index(name='report_count')
pivot = reports_count_p_y.pivot_table(index='year', columns='reporte', values='report_count')
pivot.fillna(0, inplace=True)

# Make a copy of your table
df = pivot.copy()
keep = ['Fuga', 'Falta de agua']
df['Otro'] = df.drop(columns=keep).sum(axis=1)

# Keep only desired + 'Otros'
df_filtered = df[keep + ['Otro']]

# Melt the data to long format
df_long = df_filtered.reset_index().melt(id_vars='year', var_name='Tipo de Reporte', value_name='Cantidad')

# Ensure month order
import calendar
# df_long['month_name'] = pd.Categorical(df_long['month_name'], categories=list(calendar.month_name[1:]), ordered=True)

fig = px.bar(
    df_long,
    x='year',
    y='Cantidad',
    color='Tipo de Reporte',
    title='Reportes por Mes: Fuga, Falta de Agua y Otros',
    barmode='stack',
    labels={'year': 'Año'}
)

fig.update_layout(template="plotly_white")
fig.show()

In [ ]:
# Most common reports per month name
reports_count_p_y = all_reports.groupby(by=['month', 'month_name', 'reporte']).size().reset_index(name='report_count')
pivot = reports_count_p_y.pivot_table(index=['month','month_name'], columns='reporte', values='report_count')
pivot

df = pivot.copy()
keep = ['Fuga', 'Falta de agua']
df['Otro'] = df.drop(keep, axis=1).sum(axis=1)
df_filtered = df[keep + ['Otro']]

# Melt the data to long format
df_long = df_filtered.reset_index().melt(id_vars=['month', 'month_name'], var_name='Tipo de Reporte', value_name='Cantidad')

# Ensure month order
import calendar
# df_long['month_name'] = pd.Categorical(df_long['month_name'], categories=list(calendar.month_name[1:]), ordered=True)

fig = px.bar(
    df_long,
    x='month_name',
    y='Cantidad',
    color='Tipo de Reporte',
    title='Reportes por Mes: Fuga, Falta de Agua y Otros',
    barmode='stack',
    labels={'month_name': 'Año'}
)

fig.update_layout(template="plotly_white")
fig.show()

* More stressing hour

In [ ]:
all_reports

In [ ]:
# # Example: assuming your DataFrame is called df and has 'lat', 'lon' columns
# # You can also include a weight column like 'count' or 'intensity'

# fig = px.density_mapbox(
#     all_reports,
#     lat='latitud',
#     lon='longitud',
#     z=None,  # Optional: pass a weight column here
#     radius=10,  # adjust for smoother or sharper heatmap
#     center=dict(lat=all_reports['latitud'].mean(), lon=all_reports['longitud'].mean()),
#     zoom=10,
#     mapbox_style="carto-positron",  # you can change this to "stamen-terrain", "open-street-map", etc.
#     title="Heatmap of Reports",
#     # zoom=9,
#     # center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Reportes de falta de agua en las colonias de la Ciudad de México',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

* Reporte de falta de agua por colonia (mapa) (desde 2018 a 2024)

In [ ]:
all_reports.groupby

* Reporte de fuga por colonia (mapa) (desde 2018 a 2024)

## Visualization Analysis

In [ ]:
# Plot both on the same axes
fig, ax = plt.subplots(figsize=(12, 12))

# Plot first GeoDataFrame (e.g., red outlines) and fill by 'grado'
data_concentracion.plot(ax=ax, column='ID', cmap='viridis',linewidth=0.5, label='Concentration Zones', legend=True)

# Plot second GeoDataFrame (e.g., blue fills)
data_hogares_col.plot(ax=ax, facecolor="none", alpha=0.2,)

# Add legend and grid
plt.grid(True)
plt.title("Concentración Habitacional")
plt.show()

In [ ]:
data_hogares_col.plot(column='VivHab2020', figsize=(10, 10), legend=True, cmap='viridis')

In [ ]:
data_zonas_valor.plot(column="RANGOS", legend=True)